In [1]:
import sys
sys.path.append('../..')
from MPRA_predict.utils import *
from MPRA_predict.datasets import *

import tensorflow as tf
import tensorflow_hub as hub
import gc

2024-12-11 23:46:43.207865: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-11 23:46:43.215067: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733932003.222955  209374 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733932003.225299  209374 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 23:46:43.234082: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# inputs = random_onehots(length=393216, num=100)
# np.save('data/random_onehots_393216.npy', inputs)

In [4]:
model_tf = hub.load("enformer_tfhub").model

inputs = np.load('data/random_onehots_393216.npy')
print(inputs.shape)

inputs_tf =  tf.convert_to_tensor(inputs, dtype=tf.float32)

batch_size = 4

all_predictions = []
for i in tqdm(range(0, len(inputs), batch_size)):
    batch_inputs = inputs[i:i+batch_size]
    batch_predictions = model_tf.predict_on_batch(batch_inputs)
    all_predictions.append(batch_predictions['human'])

predictions = tf.concat(all_predictions, axis=0)
print(predictions.shape)  # [batch_size, 896, 5313]

I0000 00:00:1733823763.867808   27518 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22280 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:06:00.0, compute capability: 8.9


(100, 393216, 4)


100%|██████████| 25/25 [00:29<00:00,  1.16s/it]

(100, 896, 5313)


In [5]:
tf_pred = predictions.numpy()
np.save('data/tf_pred.npy', tf_pred)
tf_pred.shape

(100, 896, 5313)

In [11]:
tf.keras.backend.clear_session()
gc.collect()

# from numba import cuda
# cuda.select_device(0)
# cuda.close()
# cuda.select_device(0)

0

In [3]:
from enformer_pytorch import from_pretrained

In [4]:
def get_pred(model, test_loader, device):
    model = model.to(device)
    y_pred = []
    model.eval()
    with torch.no_grad():
        for x in tqdm(test_loader):
            x = x[0].to(device)
            output = model(x)['human']
            y_pred.append(output.cpu().numpy())
    y_pred = np.concatenate(y_pred, axis=0)
    return y_pred

In [5]:
inputs = np.load('data/random_onehots_393216.npy')
print(inputs.shape)

inputs_torch = crop_onehots(inputs, 196608)
inputs_torch = torch.tensor(inputs_torch, dtype=torch.float32)
print(inputs_torch.shape)

(100, 393216, 4)
torch.Size([100, 196608, 4])


In [10]:
set_seed(0)
trained_model_path = 'Enformer'
device = 'cuda:0'
batch_size = 4

model = from_pretrained(trained_model_path, use_tf_gamma=True)
dataset = TensorDataset(inputs_torch)
test_data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)
torch_pred = get_pred(model, test_data_loader, device)

torch.cuda.empty_cache()

np.save('data/torch_pred.npy', torch_pred)

100%|██████████| 25/25 [00:10<00:00,  2.35it/s]


In [11]:
torch_pred = np.load('data/torch_pred.npy')
tf_pred = np.load('data/tf_pred.npy')
print(torch_pred.shape, tf_pred.shape)

(100, 896, 5313) (100, 896, 5313)


In [12]:
pearsonr(torch_pred.reshape(-1), tf_pred.reshape(-1))

PearsonRResult(statistic=0.9852688832251754, pvalue=0.0)

In [4]:
# @title `get_dataset(organism, subset, num_threads=8)`
import glob
import json
import functools


# @title `get_targets(organism)`
def get_targets(organism):
  # targets_txt = f'https://raw.githubusercontent.com/calico/basenji/master/manuscripts/cross2020/targets_{organism}.txt'
  targets_txt = f'/home/shared/enformer_data/{organism}/targets.txt'
  return pd.read_csv(targets_txt, sep='\t')


def organism_path(organism):
  # return os.path.join('gs://basenji_barnyard/data', organism)
  return os.path.join('/home/shared/enformer_data', organism)


def get_dataset(organism, subset, num_threads=8):
  metadata = get_metadata(organism)
  dataset = tf.data.TFRecordDataset(tfrecord_files(organism, subset),
                                    compression_type='ZLIB',
                                    num_parallel_reads=num_threads)
  dataset = dataset.map(functools.partial(deserialize, metadata=metadata),
                        num_parallel_calls=num_threads)
  return dataset


def get_metadata(organism):
  # Keys:
  # num_targets, train_seqs, valid_seqs, test_seqs, seq_length,
  # pool_width, crop_bp, target_length
  path = os.path.join(organism_path(organism), 'statistics.json')
  with tf.io.gfile.GFile(path, 'r') as f:
    return json.load(f)


def tfrecord_files(organism, subset):
  # Sort the values by int(*).
  return sorted(tf.io.gfile.glob(os.path.join(
      organism_path(organism), 'tfrecords', f'{subset}-*.tfr'
  )), key=lambda x: int(x.split('-')[-1].split('.')[0]))


def deserialize(serialized_example, metadata):
  """Deserialize bytes stored in TFRecordFile."""
  feature_map = {
      'sequence': tf.io.FixedLenFeature([], tf.string),
      'target': tf.io.FixedLenFeature([], tf.string),
  }
  example = tf.io.parse_example(serialized_example, feature_map)
  sequence = tf.io.decode_raw(example['sequence'], tf.bool)
  sequence = tf.reshape(sequence, (metadata['seq_length'], 4))
  sequence = tf.cast(sequence, tf.float32)

  target = tf.io.decode_raw(example['target'], tf.float16)
  target = tf.reshape(target,
                      (metadata['target_length'], metadata['num_targets']))
  target = tf.cast(target, tf.float32)

  return {'sequence': sequence,
          'target': target}


In [5]:
targets_human = get_targets('human')
targets_human.head()

,index,genome,identifier,file,clip,scale,sum_stat,description
0,0,0,ENCFF833POA,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:cerebellum male adult (27 years) and mal...
1,1,0,ENCFF110QGM,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:frontal cortex male adult (27 years) and...
2,2,0,ENCFF880MKD,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:chorion
3,3,0,ENCFF463ZLQ,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:Ishikawa treated with 0.02% dimethyl sul...
4,4,0,ENCFF890OGQ,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:GM03348


In [11]:
human_dataset = get_dataset('human', 'test').batch(1).repeat()
it = iter(human_dataset)
example = next(it)
print({k: (v.shape, v.dtype) for k,v in example.items()})

{'sequence': (TensorShape([1, 131072, 4]), tf.float32), 'target': (TensorShape([1, 896, 5313]), tf.float32)}


In [ ]:
# 数据集太大,没办法一次性预测所有序列

human_dataset = human_dataset.batch(16)

predictions = []
for sample in human_dataset:
    input_data = sample['sequence']
    prediction = model.predict_on_batch(tf.expand_dims(input_data, axis=0))  # 添加 batch 维度
    predictions.append(prediction)

predictions = tf.concat(predictions, axis=0)